In [58]:
import sys
sys.path.append("C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package")
from data_preparation import DataPreparation, Analysis, multi_data_ops
import data_operations as do

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Loading data

In [26]:
path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/Lapse Rate Projections/Data/"

In [65]:
calls_data = pd.DataFrame(pd.read_csv(path+"calls.csv"))
sales_data = pd.DataFrame(pd.read_csv(path+"sales.csv"))
policy_data = pd.DataFrame(pd.read_csv(path+"policy_data.csv"))
payment_data = pd.concat([pd.DataFrame(pd.read_csv(path+"Payment_data_1.csv")),pd.DataFrame(pd.read_csv(path+"Payment_data_2.csv"))])
lapse_data = pd.concat([pd.DataFrame(pd.read_csv(path+"lapse_1.csv")),pd.DataFrame(pd.read_csv(path+"lapse_2.csv"))])


In [ ]:
sales_data

,policy_name,call_id,contact_indicator,datetime_start
0,MQ22-0000002,fa3ff83f-39ed-4e47-bf4b-08cc517a3a02,1,2022-08-31 06:07:16
1,MQ22-0000002,86f9ce30-dad1-4d94-8593-70eaf788ff78,1,2022-08-31 06:51:27
2,ML22-0000003,817c8493-27db-4e6b-8f03-bcbfcd51aa28,0,2022-08-31 10:23:12
3,ML22-0000003,e5af692f-1fcf-48ee-91c6-97ae7d8f9367,0,2022-08-31 10:24:18
4,ML22-0000014,d631ead4-a76e-46a7-8060-83b084a82ddb,1,2022-09-02 07:46:01
...,...,...,...,...
946117,ML25-0431179,42481716-dc74-4b65-92e7-c364d3640f91,1,2025-08-05 16:24:26
946118,MQ25-0428537,9485e68d-5ef3-472d-8ba8-c31cd2528adc,0,2025-08-05 16:39:34
946119,MQ25-0428177,2efe3b8c-e556-4a0e-9778-7bf32918429f,0,2025-08-05 16:46:09
946120,MQ25-0428177,45b640e9-e7d3-489b-8600-1293e6f898d5,0,2025-08-05 16:47:15


# Filtering data to relevant policies

Relevant policies are policies that have been incepted. The rpt_datamodel_lapse has incepted policies only, hence filtering on policies within that dataset

In [66]:
pol_names = list(lapse_data['policy_name'])
sales_data = sales_data[sales_data['policy_name'].isin(pol_names)]
pol_ids= list(sales_data['policy_id'])
calls_data = calls_data[calls_data['policy_name'].isin(pol_names)]
payment_data = payment_data[payment_data['policy_id'].isin(pol_ids)]
policy_data = policy_data[policy_data['policy_name'].isin(pol_names)]

In [67]:
#Make date column of data type datetime
sales_data=do.dtype_datetime(sales_data)
calls_data=do.dtype_datetime(calls_data)
payment_data=do.dtype_datetime(payment_data)
policy_data=do.dtype_datetime(policy_data)
lapse_data=do.dtype_datetime(lapse_data)


In [41]:
len(set(pol_names)),len(set(pol_ids)),sales_data['policy_id'].nunique(), calls_data['policy_name'].nunique(), payment_data['policy_id'].nunique(),policy_data['policy_name'].nunique()

(65364, 65364, 65364, 39544, 63981, 65364)

# Aggregating Calla data and Payments data

## Calls data

In [68]:
calls_data

,policy_name,call_id,contact_indicator,datetime_start
2,ML22-0000003,817c8493-27db-4e6b-8f03-bcbfcd51aa28,0,2022-08-31 10:23:12
3,ML22-0000003,e5af692f-1fcf-48ee-91c6-97ae7d8f9367,0,2022-08-31 10:24:18
4,ML22-0000014,d631ead4-a76e-46a7-8060-83b084a82ddb,1,2022-09-02 07:46:01
12,ML22-0000005,4d104dc3-ece8-4226-ac34-404d4af38a9f,0,2022-09-02 10:54:29
13,ML22-0000005,2375400c-833e-4cc2-90b1-95baa946c9c7,0,2022-09-02 12:21:04
...,...,...,...,...
946031,MWL006970368,c583828b-baa7-4455-a092-ae5145445804,0,2025-08-05 15:03:04
946062,MWL008353846,14e6fa5f-1f4b-4c79-ab59-eb564a21a49a,0,2025-08-05 15:21:30
946075,ML24-0201111,72b23171-f6da-4216-975a-75401a70d289,1,2025-08-05 15:28:17
946098,MWL007577122,499c478a-4ceb-4aa5-a3eb-7126d7054b01,1,2025-08-05 15:46:24


In [82]:
#Adding sales date to calls data to filter to calls made after sale
calls_sale_date = multi_data_ops(file_list=[],df1=calls_data, df2=policy_data)
calls_sale_date.col_intersect

{'df1 | df2': {'policy_name'}}

In [83]:
calls_sale_date.merged_df = calls_sale_date.merging(on='policy_name', df_left=calls_sale_date.df1, df_right=calls_sale_date.df2, how='left', col_right=['policy_name','sale_date'])
calls_sale_date.merged_df

,policy_name,call_id,contact_indicator,datetime_start,sale_date
0,ML22-0000003,817c8493-27db-4e6b-8f03-bcbfcd51aa28,0,2022-08-31 10:23:12,2022-08-31
1,ML22-0000003,e5af692f-1fcf-48ee-91c6-97ae7d8f9367,0,2022-08-31 10:24:18,2022-08-31
2,ML22-0000014,d631ead4-a76e-46a7-8060-83b084a82ddb,1,2022-09-02 07:46:01,2022-09-06
3,ML22-0000005,4d104dc3-ece8-4226-ac34-404d4af38a9f,0,2022-09-02 10:54:29,2022-09-02
4,ML22-0000005,2375400c-833e-4cc2-90b1-95baa946c9c7,0,2022-09-02 12:21:04,2022-09-02
...,...,...,...,...,...
338938,MWL006970368,c583828b-baa7-4455-a092-ae5145445804,0,2025-08-05 15:03:04,2019-10-15
338939,MWL008353846,14e6fa5f-1f4b-4c79-ab59-eb564a21a49a,0,2025-08-05 15:21:30,2022-04-25
338940,ML24-0201111,72b23171-f6da-4216-975a-75401a70d289,1,2025-08-05 15:28:17,2024-04-26
338941,MWL007577122,499c478a-4ceb-4aa5-a3eb-7126d7054b01,1,2025-08-05 15:46:24,2021-02-02


In [84]:
#Selecting only call records after sale date
calls_data = calls_sale_date.merged_df[calls_sale_date.merged_df['datetime_start'].dt.strftime('%Y-%m-%d')>calls_sale_date.merged_df['sale_date'].dt.strftime('%Y-%m-%d')]


In [85]:
calls_data

,policy_name,call_id,contact_indicator,datetime_start,sale_date
6,ML22-0000005,302c00a7-8968-491d-8572-b7ee66a733af,0,2022-09-05 08:29:38,2022-09-02
270,ML22-0000180,be3c6b14-e50e-4b95-b3ef-9f6fadb40949,0,2022-09-21 14:59:24,2022-09-16
274,ML22-0000244,c069d070-6a63-4f13-933c-fdd36e0a9cbf,0,2022-09-21 15:09:21,2022-09-20
341,ML22-0000098,e1deeacb-d26e-4e11-ab97-a4141ea24a19,0,2022-09-26 07:15:42,2022-09-09
345,ML22-0000098,2ce916e4-6407-4013-85c1-990026858291,1,2022-09-26 08:39:40,2022-09-09
...,...,...,...,...,...
338938,MWL006970368,c583828b-baa7-4455-a092-ae5145445804,0,2025-08-05 15:03:04,2019-10-15
338939,MWL008353846,14e6fa5f-1f4b-4c79-ab59-eb564a21a49a,0,2025-08-05 15:21:30,2022-04-25
338940,ML24-0201111,72b23171-f6da-4216-975a-75401a70d289,1,2025-08-05 15:28:17,2024-04-26
338941,MWL007577122,499c478a-4ceb-4aa5-a3eb-7126d7054b01,1,2025-08-05 15:46:24,2021-02-02


In [86]:
calls = DataPreparation(df=calls_data)

In [87]:
calls.null_df

,nulls,percentage nulls
policy_name,0,0.0
call_id,0,0.0
contact_indicator,0,0.0
datetime_start,0,0.0
sale_date,0,0.0


In [88]:
calls.uniq_df

,no. of unique entries
policy_name,35696
call_id,257087
contact_indicator,2
datetime_start,253917
sale_date,2881


In [89]:
call_reason_data = pd.DataFrame(pd.read_csv(path+"call_reason.csv"))
call_ids = list(calls_data['call_id'])
call_reason_data = call_reason_data[call_reason_data['call_id'].isin(call_ids)]

In [ ]:
call_reason_data['resolution_code'].unique()
# related to cancellation resolution codes ['retentions_policy_cancelled','policy_cancelled','unpaid_cancelled_policy']

array(['completed', 'no_contact_post_sale', 'debit_date_changed',
       'once_off_payment', 'policy_retained', 'unpaid_collect_again',
       'duplicate', 'retentions_policy_cancelled', 'cover_reduced',
       'policy_cancelled', 'no_changes_requested',
       'unreachable_no_answer', 'requested_changes_completed',
       'banking_details_changed', nan, 'testing', 'auto_close',
       'unpaid_policy_reduced', 'second_debit_scheduled', 'sale',
       'sysytem_duplicate', 'policy_reinstated', 'policy_disputed',
       'double_debit_scheduled', 'converted',
       'cannot_offer_cover_not_the_life_insured', 'no_sale_web_preferred',
       'claim_accepted', 'not_interested_no_reason', 'qa_recaptured',
       'not_interested_unreachable_after_initial_contact',
       'not_interested_does_not_need_cover', 'unreachable_bulk_close',
       'unconverted', 'claim', 'unreachable_voicemail_only',
       'web_quote_declined', 'qa_risk_absorbed', 'alternative_payer',
       'claim_repudiated', 'no_s

In [ ]:
# creating buckets for resolution code to hot-one encode it and not have too many columns

def res_code_categories(code):
    if code in ['debit_date_changed', 'cover_reduced','requested_changes_completed','banking_details_changed','alternative_payer','cover_increased','commencement_moved',
                'additional_lives_update','bmi_update','banking_details_update', 'beneficiary_update','occupation_update','personal_details_update',
                 'educational_level_update','income_brackets_update','alcohol_intake_update']:
        return "update"
    elif code in ['claim_no_docs','claim_no_claim','claim_repudiated', 'claim','claim_accepted']:
        return "claims"
    elif code in['qa_no_risk','qa_risk_absorbed',  'qa_recaptured']:
        return "QA"
    elif code in ['sent_to_retentions', 'free_cover','premium_holiday_added','unpaid_policy_reduced', 'second_debit_scheduled','once_off_payment', 'policy_retained', 'unpaid_collect_again',
                  'policy_reinstated', 'policy_disputed','double_debit_scheduled']:
        return "payment"
    elif code in ['completed', 'no_contact_post_sale','product_explanation', 'sum_assured_disclosed', 'confirm_acceptance','converted', ]:
        return "sale"
    elif code in ['duplicate', 'no_changes_requested','unreachable_no_answer', 'testing', 'auto_close', 'sale','sysytem_duplicate', 'cannot_offer_cover_not_the_life_insured', 'no_sale_web_preferred',
       'not_interested_no_reason','not_interested_unreachable_after_initial_contact','not_interested_does_not_need_cover', 'unreachable_bulk_close',
       'unconverted',  'unreachable_voicemail_only','web_quote_declined', 'no_sale_quote_expired',  'not_interested_in_a_will','not_interested_competitors_already_covered', 
       'reading_script','note_on_behalf_of_sales_agent','unreachable_hung_up',  'cannot_offer_cover_age_too_old','no_sale_unreachable_after_initial_contact', 'documents_posted',
       'web_quote_not_interested_after_follow_up', 'not_interested_irate_do_not_call', 'no_sale_price_affordability']:
        return "other"
    else:
        return np.nan
    
call_reason_data['res_code']= call_reason_data['resolution_code'].apply(res_code_categories)
call_reason_data


,policy_name,call_id,contact_indicator,datetime_start,resolution_code,code
18,ML22-0000005,302c00a7-8968-491d-8572-b7ee66a733af,0,2022-09-05 08:29:38,completed,sale
1308,ML22-0000180,be3c6b14-e50e-4b95-b3ef-9f6fadb40949,0,2022-09-21 14:59:24,completed,sale
1319,ML22-0000244,c069d070-6a63-4f13-933c-fdd36e0a9cbf,0,2022-09-21 15:09:21,completed,sale
1782,ML22-0000098,e1deeacb-d26e-4e11-ab97-a4141ea24a19,0,2022-09-26 07:15:42,completed,sale
1819,ML22-0000098,2ce916e4-6407-4013-85c1-990026858291,1,2022-09-26 08:39:40,completed,sale
...,...,...,...,...,...,...
946327,MWL006970368,c583828b-baa7-4455-a092-ae5145445804,0,2025-08-05 15:03:04,completed,sale
946364,MWL008353846,14e6fa5f-1f4b-4c79-ab59-eb564a21a49a,0,2025-08-05 15:21:30,completed,sale
946377,ML24-0201111,72b23171-f6da-4216-975a-75401a70d289,1,2025-08-05 15:28:17,completed,sale
946401,MWL007577122,499c478a-4ceb-4aa5-a3eb-7126d7054b01,1,2025-08-05 15:46:24,completed,sale


In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: calls data might introduct target leakage due to calls being made to policyholders when payments are not made or when cancellation is requested.
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

agg_dict={'# calls': pd.NamedAgg(column='call_id', aggfunc=pd.Series.nunique),
          '# calls contacted': pd.NamedAgg(column='contact_indicator', aggfunc='sum')
          }

calls.aggregation(groupby_cols=['policy_name'],cat_cols=['res_code'] ,agg_dict_not_cat_cols=agg_dict)
calls.agg_df

## Payments data

In [6]:
data = multi_data_ops(file_list=[],df1=calls_data, df2=sales_data,df3=policy_data,df4=payment_data)

In [8]:
data.col_intersect

{'df1 | df2': set(),
 'df1 | df3': {'policy_name'},
 'df1 | df4': set(),
 'df2 | df3': {'campaign_name', 'policy_id'},
 'df2 | df4': {'policy_id', 'sales_channel'},
 'df3 | df4': {'anniversary_due',
  'cancellation_effective_date',
  'cover_start_date',
  'cover_start_date_original',
  'payment_type',
  'policy_id'}}

# Aggregating calls data

In [20]:
df = pd.DataFrame({'id':['a','a','a','b','b','c','c','c','c','c'],'A':[1,1,1,2,2,3,3,3,3,3],'B':[0,0,1,1,3,1,2,3,4,5]})

In [22]:
df[df['B']>df['A']]

,id,A,B
4,b,2,3
8,c,3,4
9,c,3,5


In [ ]:
#got to merge first to get cover_start date in the calls_data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#data[calls_data['datetime_start']>policy_data['cover_start_date']]

In [ ]:
calls = DataPreparation(df=calls_data[calls_data['policy_name'].isin(pol_names)])

In [10]:
calls.null_df

,nulls,percentage nulls
policy_name,0,0.0
call_id,0,0.0
contact_indicator,0,0.0
datetime_start,0,0.0


In [11]:
calls.uniq_df

,no. of unique entries
policy_name,110054
call_id,946122
contact_indicator,2
datetime_start,923125
